In [1]:
from utilities.utilities import load_data, get_records_by_region, create_column, finalize_dataframe, get_extreme_values, create_directory_structure, save_table, save_report, pd, assign_quartile
# settings
region_column_name = 'Region'
table_name = 'security_headers_by_region'
report_name = 'security_headers_by_region'
category = 'security_headers'
column_name_to_results_global = 'Global #'
create_directory_structure()

source_df = load_data('security_headers_checker')

In [2]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458 entries, 0 to 457
Data columns (total 16 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 458 non-null    int64 
 1   region                             458 non-null    object
 2   name                               458 non-null    object
 3   category                           458 non-null    object
 4   url                                458 non-null    object
 5   strict-transport-security          458 non-null    bool  
 6   x-frame-options                    458 non-null    bool  
 7   x-content-type-options             458 non-null    bool  
 8   content-security-policy            458 non-null    bool  
 9   x-permitted-cross-domain-policies  458 non-null    bool  
 10  referrer-policy                    458 non-null    bool  
 11  clear-site-data                    458 non-null    bool  
 12  cross-or

In [3]:
# sanity dataset
headers = [
            'strict-transport-security',
            'x-frame-options',
            'x-content-type-options',
            'content-security-policy',
            'x-permitted-cross-domain-policies',
            'referrer-policy',
            'clear-site-data',
            'cross-origin-embedder-policy',
            'cross-origin-opener-policy',
            'cross-origin-resource-policy',
            'cache-control'
        ]

# Records that are null suffered an error during data collection and are therefore treated as not having security headers.
for header in headers:
    source_df.loc[source_df[header].isna(), header] = False

In [4]:
# Analysis of HEIs with at least one OWASP Recommended Security Headers by region (Pub/Pvt)

# settings
column_to_sort = 'Without Security Headers %'
sort_ascending = True
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
dfs = []
for config_item in config:
    table_name = config_item['table_name']
    hei_type = config_item['hei_type']
    columns_to_display = [region_column_name.title(), column_name_to_results_global]
    analysis_df = get_records_by_region(source_df, hei_type=hei_type)
    # create columns
    # Column creation with distribution of records without any security headers by region
    only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
    criteria = f'category == "{hei_type}" & ({only_headers_false})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers', criteria=criteria, columns_to_display=columns_to_display)
    # Column creation with distribution of records with at least one security header by region
    all_true_filter = ' & '.join([f"`{h}` == True" for h in headers])
    at_least_one_filter = ' | '.join([f"`{h}` == True" for h in headers])
    composite_filter = f"({at_least_one_filter}) & ~({all_true_filter})"
    criteria = f'category == "{hei_type}" & ({composite_filter})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header', criteria=criteria, columns_to_display=columns_to_display)
    # Column creation with distribution of records with all security header by region
    criteria = f'category == "{hei_type}" & ({all_true_filter})'
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers', criteria=criteria, columns_to_display=columns_to_display)
    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    dfs.append(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)



df_public = dfs[0].add_suffix('(pub)')
df_private = dfs[1].add_suffix('(pvt)')
df_public = df_public.rename(columns={'Region(pub)': 'Region'})
df_private = df_private.rename(columns={'Region(pvt)': 'Region'})
df_combined = df_public.merge(df_private, on='Region', how='outer')
df_combined.fillna(0, inplace=True)
# remove columns with # in the name
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('#')]
#remove columns global
df_combined = df_combined.loc[:, ~df_combined.columns.str.contains('Global')]
# remove '%' from name of columns
df_combined.columns = df_combined.columns.str.replace('%', '')


#add column with sum of best columns
df_combined['Rank'] = df_combined['With All Security Headers (pub)'] + df_combined['With All Security Headers (pvt)']
df_combined['Rank2'] = df_combined['With At Least One Security Header (pub)'] + df_combined['With At Least One Security Header (pvt)']


#order dataframe by column Rank (from highest to lowest)
df_combined = df_combined.sort_values(by=['Rank', 'Rank2'], ascending=False)
# move just row with 'Total' in column Region to the end of the dataframe. (Use pandas.concat instead of append to avoid duplicates)
df_combined = pd.concat([df_combined[df_combined['Region'] != 'Total'], df_combined[df_combined['Region'] == 'Total']])
# reset index
df_combined.reset_index(drop=True, inplace=True)
# remove column Rank
df_combined.drop(columns=['Rank', 'Rank2'], inplace=True)
#Add a column with the quartile corresponding to the position of the row, that is, considering the total of records -1 (to exclude the total row), if a row is in position 2 it should belong to the first quartile.
df_combined['Quartile'] = df_combined.index.map(lambda rank: assign_quartile(rank, len(df_combined)-1))
# moved column 'Quartile' to the second position
cols = list(df_combined.columns)
cols = [cols[0]] + [cols[-1]] + cols[1:-1]
df_combined = df_combined[cols]


save_table(df_combined, category=category, table_name='security_headers_by_region_combined')

,Region,Global #,Without Security Headers #,Without Security Headers %,With At Least One Security Header #,With At Least One Security Header %,With All Security Headers #,With All Security Headers %
0,Saarland,5,0,0.000000,5,100.000000,0,0.0
1,Sachsen-Anhalt,8,0,0.000000,8,100.000000,0,0.0
2,Thüringen,11,0,0.000000,11,100.000000,0,0.0
3,Bayern,34,1,2.941176,33,97.058824,0,0.0
4,Berlin,12,1,8.333333,11,91.666667,0,0.0
5,Brandenburg,10,1,10.000000,9,90.000000,0,0.0
6,Hamburg,10,1,10.000000,9,90.000000,0,0.0
7,Rheinland-Pfalz,15,2,13.333333,13,86.666667,0,0.0
8,Mecklenburg-Vorpommern,7,1,14.285714,6,85.714286,0,0.0
9,Niedersachsen,21,3,14.285714,18,85.714286,0,0.0


,Region,Global #,Without Security Headers #,Without Security Headers %,With At Least One Security Header #,With At Least One Security Header %,With All Security Headers #,With All Security Headers %
0,Bremen,2,0,0.000000,2,100.000000,0,0.0
1,Saarland,5,0,0.000000,5,100.000000,0,0.0
2,Sachsen-Anhalt,2,0,0.000000,2,100.000000,0,0.0
3,Schleswig-Holstein,3,0,0.000000,3,100.000000,0,0.0
4,Thüringen,2,0,0.000000,2,100.000000,0,0.0
5,Niedersachsen,16,2,12.500000,14,87.500000,0,0.0
6,Sachsen,7,1,14.285714,6,85.714286,0,0.0
7,Nordrhein-Westfalen,25,4,16.000000,21,84.000000,0,0.0
8,Bayern,12,2,16.666667,10,83.333333,0,0.0
9,Hamburg,11,2,18.181818,9,81.818182,0,0.0


In [ ]:
# Analysis of HEIs with at least one OWASP Recommended Security Headers by region

# settings
column_to_sort = 'Without Security Headers (Public) %'
sort_ascending = True
columns_to_display = [region_column_name, column_name_to_results_global]
analysis_df = get_records_by_region(source_df)

# create columns
# Column creation with distribution of records without any security headers by region
only_public = 'category == "Public"'
only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
criteria = f"{only_public} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
only_private = 'category == "Private"'
criteria = f"{only_private} & ({only_headers_false})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with at least one security header by region
all_true_filter = ' & '.join([f"`{h}` == True" for h in headers])
at_least_one_filter = ' | '.join([f"`{h}` == True" for h in headers])
composite_filter = f"({at_least_one_filter}) & ~({all_true_filter})"
criteria = f"{only_public} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({composite_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With At Least One Security Header (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Column creation with distribution of records with all security header by region
criteria = f"{only_public} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
criteria = f"{only_private} & ({all_true_filter})"
create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='With All Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

# Finalize dataframe
analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
display(analysis_df)

# save to csv
save_table(analysis_df, category=category, table_name=table_name)


In [4]:
# Report in latex
report_results = get_extreme_values(analysis_df)

hei_public_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Public) %"), ".2f")
hei_public_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Public) %"), ".2f")
hei_public_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Public) %"), ".2f")
hei_private_without_security_headers = format(report_results.get("Total").get("Without Security Headers (Private) %"), ".2f")
hei_private_with_security_headers = format(report_results.get("Total").get("With At Least One Security Header (Private) %"), ".2f")
hei_private_with_all_security_headers = format(report_results.get("Total").get("With All Security Headers (Private) %"), ".2f")

report_figure = """
\\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\\textwidth]{charts/security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers by region.}\label{fig:security-headers}
\end{figure}
"""

report = f'{report_figure}\n\n'
report += f'The data presented in Figure~\\ref{{fig:security-headers}} provides an overview of the use of security headers at \glspl{{hei}} in \countryName. According to the data, {hei_public_without_security_headers}\% of the public institutions analyzed have not implemented any security headers on their websites, while {hei_private_without_security_headers}\% of the private institutions analyzed also lack security headers.\n\n'
report += f'On a positive note, {hei_public_with_security_headers}\% of the public institutions analyzed have implemented at least one security header in their websites, and {hei_private_with_security_headers}\% of the private institutions have implemented at least one security header.\n\n'
report += f'Finally, {hei_public_with_all_security_headers}\% of the public institutions analyzed have implemented all the security headers in their websites, and {hei_private_with_all_security_headers}\% of the private institutions have implemented all the security headers.\n\n'
report += f'In terms of regional differences, private institutions in {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Private) %").get("top_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("top_regions")[2][1], ".2f")}\%), '
report += f'while public institutions in {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Public) %").get("top_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of security headers.\n\n'
report += f'In contrast, private institutions in {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get("With At Least One Security Header (Private) %").get("bottom_regions")[2][1], ".2f")}\%), '
report += f'while public institutions in {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get("With At Least One Security Header (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get("With At Least One Security Headedfddddddddddddddddr (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of security headers.\n\n'

print(report)

# save report to file txt
save_report(report=report, category=category, report_name=report_name)


\begin{figure}[htbp]
    \centering
    \includegraphics[width=0.48\textwidth]{charts/security_headers_by_region.pdf}
    \caption{Distribution of the use Security Headers by region.}\label{fig:security-headers}
\end{figure}


The data presented in Figure~\ref{fig:security-headers} provides an overview of the use of security headers at \glspl{hei} in \countryName. According to the data, 6.93\% of the public institutions analyzed have not implemented any security headers on their websites, while 16.28\% of the private institutions analyzed also lack security headers.

On a positive note, 21.54\% of the public institutions analyzed have implemented at least one security header in their websites, and 55.26\% of the private institutions have implemented at least one security header.

Finally, 0.00\% of the public institutions analyzed have implemented all the security headers in their websites, and 0.00\% of the private institutions have implemented all the security headers.

In terms of 

In [10]:
# specific security headers by region (Pub/Pvt)

# settings
column_to_sort = 'Without Security Headers %'
sort_ascending = True
config = [
    {'table_name': 'security_headers_by_region_public', 'hei_type': 'Public'},
    {'table_name': 'security_headers_by_region_private', 'hei_type': 'Private'}
]
for config_item in config:
    hei_type = config_item['hei_type']
    for header in headers:
        table_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        report_name = f'{header.lower()}_by_region_{hei_type.lower()}'
        columns_to_display = [region_column_name, column_name_to_results_global]
        analysis_df = get_records_by_region(source_df, hei_type=hei_type)

        # Column creation with distribution of records without any security headers by region
        only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
        criteria = f'category == "{hei_type}" & ({only_headers_false})'
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers', criteria=criteria, columns_to_display=columns_to_display)
        # Column creation with distribution of records with a specific security header by region
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header}", criteria=f'category == "{hei_type}"  & (`{header}` == True)', columns_to_display=columns_to_display)

        # Column creation with distribution of records without a specific security header by region
        true_at_least_one_header_except_current = ' | '.join([f"`{h}` == True" for h in headers if h != header])
        criteria = f"(`{header}` == False) & ({true_at_least_one_header_except_current})"
        create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header}", criteria=f'category == "{hei_type}" & {criteria}', columns_to_display=columns_to_display)
        display(analysis_df)

        # Finalize dataframe
        analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
        display(analysis_df)
        # save to csv
        save_table(analysis_df, category=category, table_name=table_name)


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,cross-origin-embedder-policy,cross-origin-opener-policy,cross-origin-resource-policy,cache-control,Without Security Headers #,Without Security Headers %,With strict-transport-security #,With strict-transport-security %,Without strict-transport-security #,Without strict-transport-security %
region,,,,,,,,,,,,,,,,,,,,,
Alabama,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,2.0,14.285714,5.0,35.714286,7.0,50.000000
Alaska,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,2.0,66.666667,0.0,0.000000,1.0,33.333333
Arizona,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,0.0,0.000000,4.0,100.000000,0.0,0.000000
Arkansas,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,4.0,44.444444,4.0,44.444444,1.0,11.111111
California,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,6.0,18.750000,16.0,50.000000,10.0,31.250000
Colorado,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,5.0,35.714286,1.0,7.142857,8.0,57.142857
Connecticut,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,1.0,9.090909,0.0,0.000000,10.0,90.909091
Delaware,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0.0,0.000000,0.0,0.000000,2.0,100.000000
District of Columbia,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.0,0.000000,0.0,0.000000,1.0,100.000000


,Region,Global #,Without Security Headers #,Without Security Headers %,With strict-transport-security #,With strict-transport-security %,Without strict-transport-security #,Without strict-transport-security %
0,Nevada,3,0,0.000000,2,66.666667,1,33.333333
1,Arizona,4,0,0.000000,4,100.000000,0,0.000000
2,Washington,8,0,0.000000,2,25.000000,6,75.000000
3,New Hampshire,5,0,0.000000,1,20.000000,4,80.000000
4,Rhode Island,2,0,0.000000,1,50.000000,1,50.000000
5,New Mexico,7,0,0.000000,3,42.857143,4,57.142857
6,Delaware,2,0,0.000000,0,0.000000,2,100.000000
7,District of Columbia,1,0,0.000000,0,0.000000,1,100.000000
8,Oregon,9,0,0.000000,6,66.666667,3,33.333333
9,Idaho,4,0,0.000000,3,75.000000,1,25.000000


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,cross-origin-embedder-policy,cross-origin-opener-policy,cross-origin-resource-policy,cache-control,Without Security Headers #,Without Security Headers %,With x-frame-options #,With x-frame-options %,Without x-frame-options #,Without x-frame-options %
region,,,,,,,,,,,,,,,,,,,,,
Alabama,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,2.0,14.285714,5.0,35.714286,7.0,50.000000
Alaska,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,2.0,66.666667,0.0,0.000000,1.0,33.333333
Arizona,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,0.0,0.000000,3.0,75.000000,1.0,25.000000
Arkansas,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,4.0,44.444444,3.0,33.333333,2.0,22.222222
California,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,6.0,18.750000,13.0,40.625000,13.0,40.625000
Colorado,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,5.0,35.714286,4.0,28.571429,5.0,35.714286
Connecticut,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,1.0,9.090909,1.0,9.090909,9.0,81.818182
Delaware,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0.0,0.000000,2.0,100.000000,0.0,0.000000
District of Columbia,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.0,0.000000,0.0,0.000000,1.0,100.000000


,Region,Global #,Without Security Headers #,Without Security Headers %,With x-frame-options #,With x-frame-options %,Without x-frame-options #,Without x-frame-options %
0,Nevada,3,0,0.000000,1,33.333333,2,66.666667
1,Arizona,4,0,0.000000,3,75.000000,1,25.000000
2,Washington,8,0,0.000000,5,62.500000,3,37.500000
3,New Hampshire,5,0,0.000000,4,80.000000,1,20.000000
4,Rhode Island,2,0,0.000000,1,50.000000,1,50.000000
5,New Mexico,7,0,0.000000,1,14.285714,6,85.714286
6,Delaware,2,0,0.000000,2,100.000000,0,0.000000
7,District of Columbia,1,0,0.000000,0,0.000000,1,100.000000
8,Oregon,9,0,0.000000,6,66.666667,3,33.333333
9,Idaho,4,0,0.000000,1,25.000000,3,75.000000


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,cross-origin-embedder-policy,cross-origin-opener-policy,cross-origin-resource-policy,cache-control,Without Security Headers #,Without Security Headers %,With x-content-type-options #,With x-content-type-options %,Without x-content-type-options #,Without x-content-type-options %
region,,,,,,,,,,,,,,,,,,,,,
Alabama,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,2.0,14.285714,5.0,35.714286,7.0,50.000000
Alaska,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,2.0,66.666667,0.0,0.000000,1.0,33.333333
Arizona,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,0.0,0.000000,3.0,75.000000,1.0,25.000000
Arkansas,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,4.0,44.444444,2.0,22.222222,3.0,33.333333
California,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,6.0,18.750000,13.0,40.625000,13.0,40.625000
Colorado,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,5.0,35.714286,3.0,21.428571,6.0,42.857143
Connecticut,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,1.0,9.090909,3.0,27.272727,7.0,63.636364
Delaware,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0.0,0.000000,2.0,100.000000,0.0,0.000000
District of Columbia,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0.0,0.000000,0.0,0.000000,1.0,100.000000


...

,Region,Global #,Without Security Headers #,Without Security Headers %,With cross-origin-opener-policy #,With cross-origin-opener-policy %,Without cross-origin-opener-policy #,Without cross-origin-opener-policy %
0,Alaska,2,0,0.000000,0,0.000000,2,100.000000
1,New Mexico,3,0,0.000000,0,0.000000,3,100.000000
2,Montana,4,0,0.000000,0,0.000000,4,100.000000
3,Vermont,7,0,0.000000,0,0.000000,7,100.000000
4,Delaware,3,0,0.000000,0,0.000000,3,100.000000
5,Idaho,4,0,0.000000,0,0.000000,4,100.000000
6,Rhode Island,9,0,0.000000,0,0.000000,9,100.000000
7,Hawaii,5,0,0.000000,0,0.000000,5,100.000000
8,Indiana,39,3,7.692308,0,0.000000,36,92.307692
9,Utah,11,1,9.090909,0,0.000000,10,90.909091


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,cross-origin-embedder-policy,cross-origin-opener-policy,cross-origin-resource-policy,cache-control,Without Security Headers #,Without Security Headers %,With cross-origin-resource-policy #,With cross-origin-resource-policy %,Without cross-origin-resource-policy #,Without cross-origin-resource-policy %
region,,,,,,,,,,,,,,,,,,,,,
Alabama,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,6.0,33.333333,1.0,5.555556,11,61.111111
Alaska,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0.0,0.000000,0.0,0.000000,2,100.000000
Arizona,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,2.0,11.764706,0.0,0.000000,15,88.235294
Arkansas,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,3.0,21.428571,0.0,0.000000,11,78.571429
California,133,133,133,133,133,133,133,133,133,133,...,133,133,133,133,22.0,16.541353,0.0,0.000000,111,83.458647
Colorado,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,3.0,25.000000,0.0,0.000000,9,75.000000
Connecticut,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,5.0,29.411765,1.0,5.882353,11,64.705882
Delaware,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,0.0,0.000000,0.0,0.000000,3,100.000000
District of Columbia,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,3.0,27.272727,0.0,0.000000,8,72.727273


,Region,Global #,Without Security Headers #,Without Security Headers %,With cross-origin-resource-policy #,With cross-origin-resource-policy %,Without cross-origin-resource-policy #,Without cross-origin-resource-policy %
0,Alaska,2,0,0.000000,0,0.000000,2,100.000000
1,New Mexico,3,0,0.000000,0,0.000000,3,100.000000
2,Montana,4,0,0.000000,0,0.000000,4,100.000000
3,Vermont,7,0,0.000000,0,0.000000,7,100.000000
4,Delaware,3,0,0.000000,0,0.000000,3,100.000000
5,Idaho,4,0,0.000000,0,0.000000,4,100.000000
6,Rhode Island,9,0,0.000000,0,0.000000,9,100.000000
7,Hawaii,5,0,0.000000,0,0.000000,5,100.000000
8,Indiana,39,3,7.692308,0,0.000000,36,92.307692
9,Utah,11,1,9.090909,0,0.000000,10,90.909091


,id,name,category,Global #,strict-transport-security,x-frame-options,x-content-type-options,content-security-policy,x-permitted-cross-domain-policies,referrer-policy,...,cross-origin-embedder-policy,cross-origin-opener-policy,cross-origin-resource-policy,cache-control,Without Security Headers #,Without Security Headers %,With cache-control #,With cache-control %,Without cache-control #,Without cache-control %
region,,,,,,,,,,,,,,,,,,,,,
Alabama,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,6.0,33.333333,10,55.555556,2.0,11.111111
Alaska,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,0.0,0.000000,2,100.000000,0.0,0.000000
Arizona,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,2.0,11.764706,13,76.470588,2.0,11.764706
Arkansas,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,3.0,21.428571,8,57.142857,3.0,21.428571
California,133,133,133,133,133,133,133,133,133,133,...,133,133,133,133,22.0,16.541353,92,69.172932,19.0,14.285714
Colorado,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,3.0,25.000000,6,50.000000,3.0,25.000000
Connecticut,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,5.0,29.411765,9,52.941176,3.0,17.647059
Delaware,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,0.0,0.000000,3,100.000000,0.0,0.000000
District of Columbia,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,3.0,27.272727,6,54.545455,2.0,18.181818


,Region,Global #,Without Security Headers #,Without Security Headers %,With cache-control #,With cache-control %,Without cache-control #,Without cache-control %
0,Alaska,2,0,0.000000,2,100.000000,0,0.000000
1,New Mexico,3,0,0.000000,3,100.000000,0,0.000000
2,Montana,4,0,0.000000,4,100.000000,0,0.000000
3,Vermont,7,0,0.000000,6,85.714286,1,14.285714
4,Delaware,3,0,0.000000,3,100.000000,0,0.000000
5,Idaho,4,0,0.000000,4,100.000000,0,0.000000
6,Rhode Island,9,0,0.000000,7,77.777778,2,22.222222
7,Hawaii,5,0,0.000000,3,60.000000,2,40.000000
8,Indiana,39,3,7.692308,30,76.923077,6,15.384615
9,Utah,11,1,9.090909,7,63.636364,3,27.272727


In [ ]:
# specific security headers by region

# settings
column_to_sort = 'Without Security Headers (Public) %'
sort_ascending = True


for header in headers:
    table_name = f'{header.lower()}_by_region'
    report_name = f'{header.lower()}_by_region'
    columns_to_display = [region_column_name, column_name_to_results_global]
    analysis_df = get_records_by_region(source_df)

    # Column creation with distribution of records without any security headers by region
    only_public = 'category == "Public"'
    only_headers_false = ' & '.join([f"`{h}` == False" for h in headers])
    criteria = f"{only_public} & ({only_headers_false})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Public)', criteria=criteria, columns_to_display=columns_to_display)
    only_private = 'category == "Private"'
    criteria = f"{only_private} & ({only_headers_false})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name='Without Security Headers (Private)', criteria=criteria, columns_to_display=columns_to_display)

    # Column creation with distribution of records with a specific security header by region
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header} (Public)", criteria=f"{only_public} & (`{header}` == True)", columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"With {header} (Private)", criteria=f"{only_private} & (`{header}` == True)", columns_to_display=columns_to_display)

    # Column creation with distribution of records without a specific security header by region
    true_at_least_one_header_except_current = ' | '.join([f"`{h}` == True" for h in headers if h != header])
    criteria = f"(`{header}` == False) & ({true_at_least_one_header_except_current})"
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header} (Public)", criteria=f"{only_public} & {criteria}", columns_to_display=columns_to_display)
    create_column(source_df=source_df, analysis_dataframe=analysis_df, column_name=f"Without {header} (Private)", criteria=f"{only_private} & {criteria}", columns_to_display=columns_to_display)
    display(analysis_df)

    # Finalize dataframe
    analysis_df = finalize_dataframe(dataframe=analysis_df, column_to_sort=column_to_sort, ascending=sort_ascending, columns_to_display=columns_to_display)
    display(analysis_df)
    # save to csv
    save_table(analysis_df, category=category, table_name=table_name)

    # save to latex
    # report
    report_results = get_extreme_values(analysis_df)
    hei_private_with_specific_header = format(report_results.get("Total").get(f'With {header} (Private) %'), ".2f")
    hei_private_without_specific_header = format(report_results.get("Total").get(f"Without {header} (Private) %"), ".2f")
    hei_public_with_specific_header = format(report_results.get("Total").get(f"With {header} (Public) %"), ".2f")
    hei_public_without_specific_header = format(report_results.get("Total").get(f"Without {header} (Public) %"), ".2f")

    report_figure = f"""
    \\begin{{figure}}[htbp]
        \centering
        \includegraphics[width=0.48\\textwidth]{{charts/{table_name}.pdf}}
        \caption{{Distribution of {header} header usage by region.}}\label{{fig:{header}}}
    \end{{figure}}
    """

    report = f'{report_figure}\n\n'
    report += f'The data presented in Figure~\\ref{{fig:{header}}} provides an overview of the use of {header} security headers at \glspl{{hei}} in \countryName. According to the data, {hei_public_without_specific_header}\% of the public institutions analyzed have not implemented {header} security header on their websites, while {hei_private_without_specific_header}\% of the private institutions analyzed also lack {header} security header.\n\n'
    report += f'On a positive note, {hei_public_with_specific_header}\% of the public institutions analyzed have implemented {header} security headers on their websites, while {hei_private_with_specific_header}\% of the private institutions analyzed also implemented that security header.\n\n'
    report += f'In terms of regional differences, private institutions in {report_results.get(f"With {header} (Private) %").get("top_regions")[0][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Private) %").get("top_regions")[1][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Private) %").get("top_regions")[2][0]} ({format(report_results.get(f"With {header} (Private) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of {header} header, '
    report += f'while public institutions in {report_results.get(f"With {header} (Public) %").get("top_regions")[0][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Public) %").get("top_regions")[1][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Public) %").get("top_regions")[2][0]} ({format(report_results.get(f"With {header} (Public) %").get("top_regions")[2][1], ".2f")}\%) have a higher usage of {header} security header.\n\n'
    report += f'In contrast, private institutions in {report_results.get(f"With {header} (Private) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Private) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Private) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"With {header} (Private) %").get("bottom_regions")[2][1], ".2f")}\%), '
    report += f'while public institutions in {report_results.get(f"With {header} (Public) %").get("bottom_regions")[0][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[0][1], ".2f")}\%), {report_results.get(f"With {header} (Public) %").get("bottom_regions")[1][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[1][1], ".2f")}\%), and {report_results.get(f"With {header} (Public) %").get("bottom_regions")[2][0]} ({format(report_results.get(f"With {header} (Public) %").get("bottom_regions")[2][1], ".2f")}\%) have a lower usage of security headers.\n\n'
    # save report to file
    print(report)
    save_report(report=report, category=category, report_name=report_name)